In [ ]:
#!/usr/bin/env python3

from __future__ import annotations
import time
import numpy as np
from scipy.spatial.distance import pdist
from ase.io import read
from ase.calculators.gaussian import Gaussian
from ase.data import covalent_radii, atomic_numbers
from ase.parallel import world

import gpatom.beacon.beacon as beacon
from gpatom.beacon.str_gen import RandomBranch
from gpatom.gpfp.fingerprint import FingerPrint
from gpatom.gpfp.gp import GaussianProcess
from gpatom.gpfp.prior import CalculatorPrior, RepulsivePotential
from gpatom.gpfp.atoms_gp_interface import Model
from gpatom.gpfp.avishart_hpfitter import (
    HpFitterConstantRatioParallel,
    calculate_all_distances,
)
from hpfitter.pdistributions.normal import Normal_prior


# Odd-geometry NH3
atoms0 = read("odd_ammonia_tshape.xyz")  
atoms1 = read("NH3_triang.xyz")           
for at in (atoms0, atoms1):
    at.center(vacuum=6.0)
    at.pbc = False
structure = [atoms0, atoms1]


# Gaussian calculator
def make_gaussian_calc() -> Gaussian:
    label = f"nh3_{world.rank}_{int(time.time())}"
    return Gaussian(method="HF", basis="3-21G", scf="xqc",
                    nprocshared=4, label=label)

calc_if = beacon.CalculatorInterface(calc=make_gaussian_calc)


# Random-structure generators

RandomBranch.get_random = RandomBranch.get
sgen = RandomBranch(atoms0)
rgen = sgen

rng = np.random.RandomState(77381 + world.rank)

initgen = beacon.InitatomsGenerator(
    sgen=sgen, rgen=rgen,
    nrattle=3, rattlestrength=0.25,
    nbest=3, realfmax=0.05, rng=rng
)

# GP prior

rp_rc = 0.90 * (covalent_radii[atomic_numbers['H']]
                + covalent_radii[atomic_numbers['N']])
prior = CalculatorPrior(
    RepulsivePotential('LJ', prefactor=1.0, rc=rp_rc, constant=0.0)
)

# 5 Fingerprint & Gaussian Process

fp = FingerPrint(fp_args=dict(r_cutoff=8.0, a_cutoff=4.0, aweight=1.0),
                 calc_strain=False)

gp = GaussianProcess(prior=prior,
                     hp=dict(scale=50.0, weight=1.0, noise=1e-3),
                     use_forces=True)
model = Model(gp=gp, fp=fp)

# Hyper-parameter fitter

scale_prior = Normal_prior(mu=50.0, std=2.0)

def prior_method(E, _):
    return float(np.mean(E)) if len(E) else 0.0

def scale_bounds_method(fps):
    if len(fps) < 2:
        return [0.1, 15.0]
    dists, nn = calculate_all_distances(fps)
    return [float(np.median(nn)), 10.0 * float(np.max(dists))]

class SafeHP(HpFitterConstantRatioParallel):
    def fit(self, gp_obj):
        n_train = len(getattr(gp_obj, "fps", getattr(gp_obj, "X", [])))
        super().fit(gp_obj)
        print(f"[HP] Training points: {n_train:3d}, "
              f"GP length scale: {gp_obj.hp['scale']:5.1f}, "
              f"GP weight: {gp_obj.hp['weight']:5.1f}, "
              f"GP noise: {gp_obj.hp['noise']:.2e}")

hp_optimizer = SafeHP(scale_prior=scale_prior,
                      prior_method=prior_method,
                      scale_bounds_method=scale_bounds_method)


# Covalent-radius checker

class CovalentChecker:
    def __init__(self, factor: float = 0.90):
        self.factor = factor
    def _too_close(self, atoms):
        Z = atoms.get_atomic_numbers()
        d = pdist(atoms.get_positions())
        lim = [
            self.factor * (covalent_radii[Zi] + covalent_radii[Zj])
            for i, Zi in enumerate(Z[:-1])
            for Zj in Z[i + 1 :]
        ]
        return (d < np.array(lim)).any()
    def check(self, atoms, *_, **__):
        return (False, "short bond") if self._too_close(atoms) else (True, "accepted")
    accept = check
    def check_fingerprint_distances(self, *_, **__):
        return True

checker = CovalentChecker()

# Surrogate optimiser & acquisition

surropt = beacon.SurrogateOptimizer(fmax=0.05, relax_steps=100)
acq     = beacon.LowerBound(kappa=2.0)

# BEACON

bo = beacon.BEACON(
    calculator   = calc_if,
    initatomsgen = initgen,
    init_atoms   = structure,
    model        = model,
    ninit        = 2,
    ndft         = 30,
    nsur         = 3,
    surropt      = surropt,
    acq          = acq,
    checker      = checker,
    hp_optimizer = hp_optimizer,
)

def main():
    bo.run()
    print("\nBEACON finished.")

if __name__ == "__main__":
    main()
    num_training = len(bo.model.data)
    scale = bo.model.gp.hp.get('scale', "N/A")
    print(f"\nBEACON summary: Training points: {num_training}, "
          f"Final GP length scale: {scale}")

In [1]:
# K-sweep schedule for ndft 30steps

# BEACON 
bo = beacon.BEACON(
    calculator=calc_if,
    initatomsgen=initgen,
    init_atoms=structure,
    model=model,
    ninit=2,
    ndft=30,
    nsur=3,
    surropt=surropt,
    acq=acq,
    checker=checker,
    hp_optimizer=hp_optimizer,
)

# Kappa Sweep Phases
PHASES = [
    ('explore κ=3.0', 3.0, 10),
    ('explore κ=1.0', 1.0, 10),
    ('explore κ=0.0', 0.0, 10),
]


# Main:
def main():
    total = 0
    for phase, kappa, nsteps in PHASES:
        bo.acq = beacon.LowerBound(kappa=kappa)
        for step in range(nsteps):
            total += 1
            print(f"=== Step {total:02d}/60 — phase: {phase} (κ={kappa}) ===")
            bo.ndft = total 
            bo.run()
    print(f"All phases complete — {total} evaluations total.")

    num_training = len(bo.model.data)
    try:
        scale = bo.model.gp.hp['scale']
    except Exception:
        scale = "N/A"
    print(f"\nBEACON summary: Training points: {num_training}, Final GP length scale: {scale}")

if __name__ == "__main__":
    main()



